In [ ]:
# отключим предупреждения Anaconda
import warnings
warnings.simplefilter('ignore')

# будем отображать графики прямо в jupyter'e
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
#графики в svg выглядят более четкими
%config InlineBackend.figure_format = 'svg' 

#увеличим дефолтный размер графиков
from pylab import rcParams
rcParams['figure.figsize'] = 8, 5
import pandas as pd

In [ ]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly
import plotly.graph_objs as go

init_notebook_mode(connected=True)

In [ ]:
#10 vertical divs
#25 subdivs
#2500 points verical
#2**16 values
#0.75 V/div
#7.5 p/p
#-3.75 - 3.75 volts
#-32768 - 32767 data
#(x - in_min) * (out_max - out_min) / (in_max - in_min) + out_min;
#add 0.75 volts since zero level below zero
def data_to_volts(x):
    return(x + 32768) * 7.5 / 2**16 - 3.75
#using opamp with negative feedback
#and memr as low-side resistor    

In [ ]:
from datetime import datetime as dt
data_fall=[]
time_fall=[]
#test_fall_0_05#04#2019#14#21#08.txt
folder="05#04#2019#14#21#08"
for i in range(5):
    file_data=[]
    file_times=[]
    with open("{}/test_fall_{}_{}.txt".format(folder,i,folder), "r") as f:        
        file_data=eval(f.readline())
        file_times=eval(f.readline())
    data_fall.append(file_data)
    time_fall.append([dt.strptime(s,"%d#%m#%Y#%H#%M#%S#%f") for s in file_times])

In [ ]:
time_delta_fall=[[(i-t[0]).total_seconds() for i in t]for t in time_fall]

In [ ]:
data_grow=[]
time_grow=[]
#test_grow_0_05#04#2019#14#21#08.txt
folder="05#04#2019#14#21#08"
for i in range(5):
    file_data=[]
    file_times=[]
    with open("{}/test_grow_{}_{}.txt".format(folder,i,folder), "r") as f:        
        file_data=eval(f.readline())
        file_times=eval(f.readline())
    data_grow.append(file_data)
    time_grow.append([dt.strptime(s,"%d#%m#%Y#%H#%M#%S#%f") for s in file_times])
time_delta_grow=[[(i-t[0]).total_seconds() for i in t]for t in time_grow]

In [ ]:
import numpy as np
import pandas as pd

# df2_fall_only_filt=[]
# for d in df2_fall_filt:
#     df2_fall_only_filt.append()

df_grow=[]#dataframes for basic pictures
df2_grow=[]#dataframes for screenshot to screenshot pictures
for i in data_grow:
    npdata=np.array(i)
    df_grow.append(pd.DataFrame(npdata.transpose())[1:])
    #df2.append(pd.DataFrame(npdata))

In [ ]:
df_fall=[]#dataframes for basic pictures
df2_fall=[]#dataframes for screenshot to screenshot pictures
for i in range(len(data_fall)):
    npdata=np.array(data_fall[i])
    print(len(data_fall[i]))
    d=pd.DataFrame(npdata.transpose()).drop(columns=list(range(df_grow[i].shape[1])))
    print(df_grow[i].shape)
    print(d.shape)    
    name=d.columns[0]
    d=d.rename(lambda x: x-name, axis='columns')    
    df_fall.append(d)
    #df2.append(d.transpose())

In [ ]:
for i in range(len(df_grow)):    
    time_delta_fall[i]=time_delta_fall[i][len(time_delta_grow[i]):]    

In [ ]:
datas =[go.Scatter(
            x=df_grow[0].index,
            y=df_grow[0][i],
            name='{}'.format(time_delta_grow[0][i])
            ) for i in range(1,len(time_delta_grow[0]),50)]
    
layout = go.Layout(
    title='Raw oscilloscope snapshots - data{}'.format(0),
    xaxis=dict(
        title="TICKS",
        showgrid=True,
        zeroline=True,
        showline=True,
        mirror='ticks',
        gridcolor='#bdbdbd',
        gridwidth=2,
        zerolinecolor='#969696',
        zerolinewidth=4,
        linecolor='#636363',
        linewidth=6
    ),
    yaxis=dict(
        title="POINTS",
        showgrid=True,
        zeroline=True,
        showline=True,
        mirror='ticks',
        gridcolor='#bdbdbd',
        gridwidth=2,
        zerolinecolor='#969696',
        zerolinewidth=4,
        linecolor='#636363',
        linewidth=6
    )
)

# cоздаем объект Figure и визуализируем его
fig = go.Figure(data=datas, layout=layout)
iplot(fig, show_link=False)

In [ ]:
for i in range(len(df_fall)):
    print(df_fall[i].shape)
    print(len(time_delta_fall[i]))

In [ ]:
#convert ticks to times
#convert points to voltage
for i in range(len(df_grow)):
    df_grow[i].index=df_grow[i].index*20*10**-6
    df_grow[i]=df_grow[i].applymap(data_to_volts)
    df2_grow.append(df_grow[i])
for i in range(len(df_fall)):
    df_fall[i].index=df_fall[i].index*20*10**-6
    df_fall[i]=df_fall[i].applymap(data_to_volts)
    df2_fall.append(df_fall[i])

In [ ]:
for dfidx in range(len(df_grow)):
    datas =[go.Scatter(
            x=df_grow[dfidx].index,
            y=df_grow[dfidx][i],
            name='{}'.format(time_delta_grow[dfidx][i])
            ) for i in range(1,len(time_delta_grow[dfidx]),50)]
    layout = go.Layout(
        title='Raw oscilloscope snapshots - data{}'.format(dfidx),
        xaxis=dict(
            title="TIME",
            showgrid=True,
            zeroline=True,
            showline=True,
            mirror='ticks',
            gridcolor='#bdbdbd',
            gridwidth=2,
            zerolinecolor='#969696',
            zerolinewidth=4,
            linecolor='#636363',
            linewidth=6
        ),
        yaxis=dict(
            title="VOLTS",
            showgrid=True,
            zeroline=True,
            showline=True,
            mirror='ticks',
            gridcolor='#bdbdbd',
            gridwidth=2,
            zerolinecolor='#969696',
            zerolinewidth=4,
            linecolor='#636363',
            linewidth=6
        )
    )

    # cоздаем объект Figure и визуализируем его
    fig = go.Figure(data=datas, layout=layout)
    iplot(fig, show_link=False)

In [ ]:
df_grow_filt=[]
df2_grow_filt=[]
for d in df_grow:
    temp=d.sub(d[0], axis='rows')
    temp=d.rolling(200,win_type="gaussian").mean(std=3)    
    df_grow_filt.append(temp)
    df2_grow_filt.append(temp.transpose())
df_fall_filt=[]
df2_fall_filt=[]
for d in df_fall:
    temp=d.sub(d[0], axis='rows')
    temp=d.rolling(200,win_type="gaussian").mean(std=3)    
    df_fall_filt.append(temp)
    df2_fall_filt.append(temp.transpose())

In [ ]:
print(df_fall_filt[1].shape)

In [ ]:
for dfidx in range(len(df_fall_filt)):
    datas =[go.Scatter(
            x=df_fall_filt[dfidx].index,
            y=df_fall_filt[dfidx][i],
            name='{}'.format(time_delta_fall[dfidx][i])
            ) for i in range(1,len(time_delta_fall[dfidx]),50)]
    layout = go.Layout(
        title='Filtered oscilloscope snapshots - data{}'.format(dfidx),
        xaxis=dict(
            title="TIME",
            showgrid=True,
            zeroline=True,
            showline=True,
            mirror='ticks',
            gridcolor='#bdbdbd',
            gridwidth=2,
            zerolinecolor='#969696',
            zerolinewidth=4,
            linecolor='#636363',
            linewidth=6
        ),
        yaxis=dict(
            title="VOLTAGE",
            showgrid=True,
            zeroline=True,
            showline=True,
            mirror='ticks',
            gridcolor='#bdbdbd',
            gridwidth=2,
            zerolinecolor='#969696',
            zerolinewidth=4,
            linecolor='#636363',
            linewidth=6
        )
    )

    # cоздаем объект Figure и визуализируем его
    fig = go.Figure(data=datas, layout=layout)
    iplot(fig, show_link=False)

In [ ]:
point_list_grow=[0.0072,0.0172,0.0272,0.0372,0.0472]
point_list_fall=[0.005,0.015,0.025,0.035,0.045]

In [ ]:
for dfidx in range(len(point_list_grow)):
    datas =[go.Scatter(
            x=time_delta_grow[i],
            y=df2_grow_filt[i][point_list_grow[dfidx]],
            name='grow_{}'.format(i)
            ) for i in range(len(df2_grow_filt))]+[go.Scatter(
            x=time_delta_fall[i],
            y=df2_fall_filt[i][point_list_fall[dfidx]],
            name='fall_{}'.format(i)
            ) for i in range(len(df2_fall_filt))]
    layout = go.Layout(
        title='Volts change at sigle screenshot time - data{}'.format(dfidx),
        xaxis=dict(
            title="TIME",
            showgrid=True,
            zeroline=True,
            showline=True,
            mirror='ticks',
            gridcolor='#bdbdbd',
            gridwidth=2,
            zerolinecolor='#969696',
            zerolinewidth=4,
            linecolor='#636363',
            linewidth=6
        ),
        yaxis=dict(
            title="VOLTAGE",
            showgrid=True,
            zeroline=True,
            showline=True,
            mirror='ticks',
            gridcolor='#bdbdbd',
            gridwidth=2,
            zerolinecolor='#969696',
            zerolinewidth=4,
            linecolor='#636363',
            linewidth=6
        )
    )

    # cоздаем объект Figure и визуализируем его
    fig = go.Figure(data=datas, layout=layout)
    iplot(fig, show_link=False)

In [ ]:
def volts_to_res(u_signal,u_res):
    return u_signal*1.2*10**6/u_res

In [ ]:
df2_grow_resistance = []
for d in df2_grow_filt:
    df2_grow_resistance.append(d.applymap(lambda x:volts_to_res(0.8,x)))
df2_fall_resistance = []
for d in df2_fall_only_filt:
    df2_fall_resistance.append(d.applymap(lambda x:volts_to_res(-0.2,x)))

In [ ]:
for dfidx in range(len(point_list_grow)):    
    datas =[go.Scatter(
            x=time_delta_grow[i],
            y=df2_grow_resistance[i][point_list_grow[dfidx]],
            name='grow_{}'.format(i)
            ) for i in range(len(df2_grow_resistance))]+[go.Scatter(
            x=time_delta_fall[i],
            y=df2_fall_resistance[i][point_list_fall[dfidx]],
            name='fall_{}'.format(i)
            ) for i in range(len(df2_fall_resistance))]
    layout = go.Layout(
        title='Conductance change at sigle screenshot time - data{}'.format(dfidx),
        xaxis=dict(
            title="TIME",
            showgrid=True,
            zeroline=True,
            showline=True,
            mirror='ticks',
            gridcolor='#bdbdbd',
            gridwidth=2,
            zerolinecolor='#969696',
            zerolinewidth=4,
            linecolor='#636363',
            linewidth=6
        ),
        yaxis=dict(
            title="SIEMENSES",
            showgrid=True,
            zeroline=True,
            showline=True,
            mirror='ticks',
            gridcolor='#bdbdbd',
            gridwidth=2,
            zerolinecolor='#969696',
            zerolinewidth=4,
            linecolor='#636363',
            linewidth=6
        )
    )

    # cоздаем объект Figure и визуализируем его
    fig = go.Figure(data=datas, layout=layout)
    iplot(fig, show_link=False)

In [ ]:
for dfidx in point_list:
    datas =[go.Scatter(
            x=time_delta_fall[i],
            y=df2_fall_resistance[i][dfidx],
            name='fall_{}'.format(i)
            ) for i in range(len(df2_fall_resistance))]
    layout = go.Layout(
        title='Resistance change at sigle screenshot time - data{}'.format(dfidx),
        xaxis=dict(
            title="TIME",
            showgrid=True,
            zeroline=True,
            showline=True,
            mirror='ticks',
            gridcolor='#bdbdbd',
            gridwidth=2,
            zerolinecolor='#969696',
            zerolinewidth=4,
            linecolor='#636363',
            linewidth=6
        ),
        yaxis=dict(
            title="OHMS",
            showgrid=True,
            zeroline=True,
            showline=True,
            mirror='ticks',
            gridcolor='#bdbdbd',
            gridwidth=2,
            zerolinecolor='#969696',
            zerolinewidth=4,
            linecolor='#636363',
            linewidth=6
        )
    )

    # cоздаем объект Figure и визуализируем его
    fig = go.Figure(data=datas, layout=layout)
    iplot(fig, show_link=False)